In [7]:
import os
import unittest
import requests
import urllib3
from utils_demo import *
from loguru import logger
from unittest.mock import patch
from pprint import pprint
from os.path import dirname, abspath
import warnings
warnings.filterwarnings('ignore')
from performance_data.performance_data import PhaseData, DataTable
from performance_data import DATASET_FILE
from app_decomposer import DEFAULT_CONFIGURATION, KIWI_CONFIG, CURRENT_DIR, API_DICT_TS, IOI_SAMPLING_PERIOD, DATASET_SOURCE
from app_decomposer.api_connector import request_delegator
from app_decomposer.config_parser import Configuration
from app_decomposer.api_connector import TimeSeries
from loguru import logger

In [15]:
#%%capture
logger.remove()
def decompose_ioi_job(jobid):
    with patch.object(ComplexDecomposer, 'get_job_timeseries') as mock_get_timeseries:
        with patch.object(Configuration, 'get_kc_token') as mock_get_kc_token:
            with patch.object(ComplexDecomposer, 'get_job_node_count') as mock_get_node_count:                    
                mock_get_timeseries.return_value = get_job_timeseries_from_file(job_id=jobid)
                mock_get_kc_token.return_value = 'token'
                mock_get_node_count.return_value = 1
                # init the job decomposer
                #cd = ComplexDecomposer(v0_threshold=0.02)
                cd = ComplexDecomposer()
                return cd
        
# Launch decomposition on the signal
cd = decompose_ioi_job(jobid=3917)
print(cd)
representation = cd.get_job_representation(merge_clusters=True)
# This is the app encoding representation for Execution Simulator
print(f"compute={representation['events']}, reads={representation['read_volumes']}, read_bw={representation['read_bw']}")
print(f"writes={representation['write_volumes']}, write_bw={representation['write_bw']}")
# Normalize signals to seconds and MB
timestamps = (cd.timestamps.flatten() - cd.timestamps.flatten()[0])/5
original_read =  cd.read_signal.flatten()/1e6
original_write = cd.write_signal.flatten()/1e6



compute=[0, 1, 4, 9, 11, 28, 42, 46, 48], reads=[0, 1731312012, 3169488157, 1585247951, 0, 0, 0, 0, 0], read_bw=[0, 1731312012.0, 1584744078.5, 1585247951.0, 0, 0, 0, 0, 0]
writes=[0, 0, 4139017218, 0, 1901214425, 5334798849, 5334798849, 5334798849, 0], write_bw=[0, 0, 1034754304.5, 0, 950607212.5, 1066959769.8, 1066959769.8, 1333699712.25, 0]


In [16]:
events = representation['events']
reads = representation['read_volumes']
writes = representation['write_volumes']
# reads = list(map(lambda x, y: x if x > y else 0, representation['read_volumes'], representation["write_volumes"]))
# writes = list(map(lambda x, y: x if x > y else 0, representation['write_volumes'], representation["read_volumes"]))
print(events)
print(reads)
print(writes)

[0, 1, 4, 9, 11, 28, 42, 46, 48]
[0, 1731312012, 3169488157, 1585247951, 0, 0, 0, 0, 0]
[0, 0, 4139017218, 0, 1901214425, 5334798849, 5334798849, 5334798849, 0]
